In [1]:
import copy
import pandas as pd
from bokeh.layouts import row, widgetbox
from bokeh.models import Jitter
from bokeh.models import Select, CustomJS, Button
from bokeh.palettes import plasma
from bokeh.plotting import curdoc, figure, ColumnDataSource
from bokeh.tile_providers import STAMEN_TERRAIN

default_color_count = 11
SIZES = list(range(6, 22, 3))

In [2]:
def get_data():
    data = pd.read_csv("./bcur_munged.csv")

    data['grp'] = data['grp'].apply(str)
    data['assign'] = data['assign'].apply(str)

    data.northing = data.northing.apply(lambda x: 15000000 if pd.isnull(x) else x)
    data.easting = data.easting.apply(lambda x: 0 if pd.isnull(x) else x)

    data = data.applymap(lambda x: "NaN" if pd.isnull(x) else x)

    return data


def create_source():
    df["size"] = 9
    if size.value != 'None':
        groups = pd.cut(df[size.value].values, len(SIZES))
        df["size"] = [SIZES[xx] for xx in groups.codes]

        df["color"] = "#31AADE"
    if color.value != 'None' and color.value in quantileable:
        colors = plasma(default_color_count)
        groups = pd.cut(df[color.value].values, len(colors))
        df["color"] = [colors[xx] for xx in groups.codes]
    elif color.value != 'None' and color.value in discrete_colorable:
        values = df[color.value][pd.notnull(df[color.value])].unique()
        colors = plasma(len(values))
        if df[color.value].dtype == object and all([val.isnumeric() for val in values]):
            values = sorted(values, key=lambda x: float(x))
        codes = dict(zip(values, range(len(values))))
        groups = [codes[val] for val in df[color.value].values]
        df["color"] = [colors[xx] for xx in groups]

    df["ns"] = df["northing"]
    df["es"] = df["easting"]

    # create a ColumnDataSource from the  data set
    return ColumnDataSource(df)


def update_source(s):
    df["size"] = 9
    if size.value != 'None':
        groups = pd.cut(df[size.value].values, len(SIZES))
        df["size"] = [SIZES[xx] for xx in groups.codes]

        df["color"] = "#31AADE"
    if color.value != 'None' and color.value in quantileable:
        colors = plasma(default_color_count)
        groups = pd.cut(df[color.value].values, len(colors))
        df["color"] = [colors[xx] for xx in groups.codes]
    elif color.value != 'None' and color.value in discrete_colorable:
        values = df[color.value][pd.notnull(df[color.value])].unique()
        colors = plasma(len(values))
        if df[color.value].dtype == object and all([val.isnumeric() for val in values]):
            values = sorted(values, key=lambda x: float(x))
        codes = dict(zip(values, range(len(values))))
        groups = [codes[val] for val in df[color.value].values]
        df["color"] = [colors[xx] for xx in groups]

    df["ns"] = df["northing"]
    df["es"] = df["easting"]

    # create a ColumnDataSource from the  data set
    s.data = s.from_df(df)


def create_crossfilter(s):
    kw = dict()
    if x.value in discrete:
        values = df[x.value][pd.notnull(df[x.value])].unique()
        if df[color.value].dtype == object and all([val.isnumeric() for val in values]):
            kw["x_range"] = sorted(values, key=lambda x: float(x))
        else:
            kw["x_range"] = sorted(values)
    if y.value in discrete:
        values = df[y.value][pd.notnull(df[y.value])].unique()
        if df[color.value].dtype == object and all([val.isnumeric() for val in values]):
            kw["y_range"] = sorted(values, key=lambda x: float(x))
        else:
            kw["y_range"] = sorted(values)

    x_title = x.value.title()
    y_title = y.value.title()

    p = figure(plot_height=600, plot_width=800, tools="wheel_zoom,reset,box_select", **kw,
               title="%s vs %s" % (x_title, y_title))

    if x.value in discrete:
        p.xaxis.major_label_orientation = pd.np.pi / 4

    # plot data on crossfilter
    p.circle(x=x.value, y=y.value, color="color", size="size", source=s, line_color="white",
             alpha=0.6)

    return p


def create_map(s):
    stamen = copy.copy(STAMEN_TERRAIN)
    # create map
    bound = 20000000  # meters
    m = figure(tools="wheel_zoom,reset,box_select", x_range=(-bound, bound),
               y_range=(-bound, bound))
    m.axis.visible = False
    m.add_tile(stamen)

    # plot data on world map
    m.circle(x="es", y="ns", color="color", size="size", source=s, line_color="white",
             alpha=0.6,
             hover_color='white', hover_alpha=0.5)

    return m


def create_jitter_buttons(s):
    map_jitter = Jitter(width=16093, distribution="uniform")

    jitter_callback = CustomJS(args=dict(source=s, map_jitter=map_jitter), code="""
        var data = source.data;
        for (var i = 0; i < data['easting'].length; i++) {
            data['es'][i] = map_jitter.compute(data['easting'][i]);
        }
        for (var i = 0; i < data['northing'].length; i++) {
            data['ns'][i] = map_jitter.compute(data['northing'][i]);
        }
        source.trigger('change');
    """)

    reset_jitter_callback = CustomJS(args=dict(source=s, map_jitter=map_jitter), code="""
        var data = source.data;
        for (var i = 0; i < data['easting'].length; i++) {
            data['es'][i] = data['easting'][i];
        }
        for (var i = 0; i < data['northing'].length; i++) {
            data['ns'][i] = data['northing'][i];
        }
        source.trigger('change');
    """)

    map_jitter_button = Button(label='apply jitter to map', callback=jitter_callback)

    reset_map_jitter_button = Button(label='remove jitter from map',
                                     callback=reset_jitter_callback)

    return map_jitter_button, reset_map_jitter_button


# callbacks
def x_change(attr, old, new):
    update_source(source)
    layout.children[1] = create_crossfilter(source)


def y_change(attr, old, new):
    update_source(source)
    layout.children[1] = create_crossfilter(source)


def size_change(attr, old, new):
    update_source(source)


def color_change(attr, old, new):
    update_source(source)

In [3]:
# load data
df = get_data()

In [4]:
# catigorize columns
columns = list(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
discrete_colorable = [x for x in discrete if len(df[x].unique()) <= max(len(df["grp"].unique()),
                                                                        len(df[
                                                                                "assign"].unique()))]
continuous = [x for x in columns if x not in discrete]
quantileable = [x for x in continuous if len(df[x].unique()) > 20]

In [5]:
# create widgets
x = Select(title='X-Axis', value='LD1', options=columns)
x.on_change('value', x_change)

y = Select(title='Y-Axis', value='LD2', options=columns)
y.on_change('value', y_change)

size = Select(title='Size', value='posterior_assign', options=['None'] + quantileable)
size.on_change('value', size_change)

color = Select(title='Color', value='assign', options=['None'] + quantileable + discrete_colorable)
color.on_change('value', color_change)

In [6]:
# initilize source
source = create_source()

In [7]:
# initialize plots
crossfilter = create_crossfilter(source)
map = create_map(source)

In [8]:
source.data['Decimal Latitude']

['NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 12.53593,
 12.53593,
 12.53593,
 12.53593,
 12.53593,
 12.569162,
 12.569162,
 12.569162,
 12.569162,
 12.569162,
 12.569162,
 12.569162,
 12.569162,
 14.497401000000002,
 14.497401000000002,
 14.497401000000002,
 14.497401000000002,
 14.497401000000002,
 21.375116000000002,
 21.375116000000002,
 21.375116000000002,
 21.375116000000002,
 21.393589000000002,
 21.393589000000002,
 21.68204444430874,
 21.68204444430874,
 21.68204444430874,
 21.674092035256329,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 20.964532999999999,
 11.653366999999999,
 11.653366999999999,
 11.653366999999999,
 11.653366999999999,
 19.814882999999998,
 19.814882999999998,
 19.81

In [16]:
df["Decimal Latitude"] = df["Decimal Latitude"].astype("float64")
df["Decimal Longitude"] = df["Decimal Longitude"].astype("float64")

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
6             NaN
7             NaN
8             NaN
9             NaN
10            NaN
11            NaN
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17            NaN
18            NaN
19            NaN
20            NaN
21            NaN
22            NaN
23            NaN
24            NaN
25            NaN
26            NaN
27     -16.355377
28     -16.355377
29     -16.355377
          ...    
194    105.765926
195    105.765926
196    108.277199
197     83.714960
198     83.714960
199     83.714960
200     83.714960
201    100.992541
202    100.992541
203    100.992541
204    100.992541
205    100.992541
206    100.992541
207    100.992541
208    100.992541
209    100.992541
210    121.774017
211    121.774017
212    121.774017
213    121.774017
214    121.774017
215    120.922000
216    120.922000
217    120.922000
218    120

In [13]:
df.dtypes

key                       object
LD1                      float64
LD2                      float64
LD3                      float64
LD4                      float64
LD5                      float64
LD6                      float64
LD7                      float64
LD8                      float64
LD9                      float64
LD10                     float64
LD11                     float64
LD12                     float64
LD13                     float64
assign                    object
grp                       object
posterior_assign         float64
posterior_grp            float64
MS                         int64
Country/Territory         object
Island/Province/State     object
Locality                  object
Decimal Latitude         float64
Decimal Longitude         object
Elevation                 object
Date collected            object
Collector                 object
Attractant                object
Sex                       object
Comments                  object
Comments 2

In [18]:
import colorcet as cc